In [1]:
import os
import cv2
from tqdm import tqdm
from random import sample
from PIL import Image
import pickle
import time
import sys

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset

from sklearn.metrics import accuracy_score
from sklearn import svm, metrics, preprocessing


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pip install facenet-pytorch
from facenet_pytorch import MTCNN # pretrained model for image recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.7 MB/s eta 0:00:00


In [3]:
from shutil import copyfile

copyfile(src = "../input/mtcnn-model/facial_analysis_kaggle.py", dst = "../working/facial_analysis_kaggle.py")
copyfile(src = "../input/mtcnn-model/mtcnn.pb", dst = "../working/mtcnn.pb")

'../working/mtcnn.pb'

In [4]:
from facial_analysis_kaggle import FacialImageProcessing

imgProcessing = FacialImageProcessing(False)
mtcnn = MTCNN(keep_all=True, min_face_size=40, device='cpu')

/kaggle/working/mtcnn.pb


In [5]:
def extract_and_save_faces(data_path, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for person in tqdm(os.listdir(data_path)):
        person_path = data_path + '/' + person
        # print(person_path)
        if not os.path.exists(save_dir + '/' + person):
            os.makedirs(save_dir + '/' + person)
        for video_name in os.listdir(person_path):
            video_format = list(os.listdir(person_path + '/' + video_name))[0].split('.')[-1]
            video_path = person_path + '/' + video_name + '/' + video_name + '.' + video_format
            video = cv2.VideoCapture(video_path)
            fps = video.get(cv2.CAP_PROP_FPS)
            images = []
            filenames = []
            i = 0.
            stride = 1.0
            
            frame_times = []

            while video.isOpened():
                ret, frame = video.read()
                if ret:
                    images.append(frame)
                    frame_times.append(i)
                    i += stride
                    video.set(1, round(i * fps))
                else:
                    video.release()
                    break
                    
            for i, image in enumerate(images):
                bounding_boxes, _ = imgProcessing.detect_faces(image)
                # bounding_boxes, _, _ = mtcnn.detect(image, landmarks=True)
                
                if bounding_boxes is None or len(bounding_boxes)==0:
                    # print('No faces found for ',video_path + '_' + str(i+1))
                    face_img = image
                    faceFound='noface'
                else:
                    if len(bounding_boxes)>1:
                        # print('Too many faces (',len(bounding_boxes),') found for ',filename)
                        bounding_boxes=bounding_boxes[:1]
                    
                    b=[int(bi) for bi in bounding_boxes[0]]
                    x1,y1,x2,y2=b[0:4]
                    face_img=image[y1:y2,x1:x2,:]
                    
                    if np.prod(face_img.shape)==0:
                        # print('Empty face ',b,' found for ',filename)
                        continue
                        
                    faceFound=''

                # print('/kaggle/working/daisee_test/' + person + '/' + video_name + '_' + str(i+1) + '_' + faceFound + '.png')
                cv2.imwrite(save_dir + '/' + person + '/' + video_name + '_' + str(i+1) + '_' + faceFound + '.png', face_img)
                # cv2.imwrite(os.path.join(save_path, folder, root+faceFound+ext), face_img)

In [6]:
train_video_path = r'/kaggle/input/daisee/DAiSEE/DataSet/Train'
val_video_path = r'/kaggle/input/daisee/DAiSEE/DataSet/Validation'
test_video_path = r'/kaggle/input/daisee/DAiSEE/DataSet/Test'

In [7]:
extract_and_save_faces(train_video_path, r'/kaggle/working/daisee_train')

 24%|██▍       | 17/70 [24:15<1:23:21, 94.36s/it][mpeg4 @ 0x59d9ad6c0100] I cbpy damaged at 16 8
[mpeg4 @ 0x59d9ad6c0100] Error at MB: 344
100%|██████████| 70/70 [1:36:25<00:00, 82.65s/it]


In [8]:
extract_and_save_faces(val_video_path, r'/kaggle/working/daisee_val')

100%|██████████| 22/22 [28:03<00:00, 76.52s/it]


In [9]:
# extract_and_save_faces(test_video_path)

Что нужно для качественного завершения вкр:
- лица DAiSEE всех трех выборок
- определяем точку, куда в основном смотрят чуваки разных классов на DAiSEE 
- определяем их поворот по x и y
- определяем средний ratio глаза (проверить на наличие азиатов)

Так понимаем на train понимаем, какие признаки у каких классов в какую сторону смещены.
Затем подбираем пороговые значения, делаем классификацию, замеряем метрики

In [10]:
#import os
#os.chdir(r'/kaggle/working')

#!tar -czf daisee_test_faces.tar.gz daisee_test

#from IPython.display import FileLink

# FileLink(r'daisee_test_faces.tar.gz')

In [11]:
# import shutil
# shutil.rmtree('/kaggle/working/daisee_test')